In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from numpy.fft import fft

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (30, 20)

## process_file() takes a string with a path to the .csv
def process_file(pathname):
    res = pd.read_csv(pathname, sep = ',', header = 0, index_col = None, names = ['time', 'seconds', 'z', 'y', 'x'])
    return res

In [7]:
## Importing Data

## Walking
d_rp5min = process_file('processed_data/drp_5min_bw.csv')
d_rp5min['movement'] = 'walking'
d_walk_split = np.array_split(d_rp5min, 120)

m_rp5min = process_file('processed_data/mrp_5min_bw.csv')
m_rp5min['movement'] = 'walking'
m_walk_split = np.array_split(m_rp5min, 120)

s_rp5min = process_file('processed_data/srp_5min_bw.csv')
s_rp5min['movement'] = 'walking'
s_walk_split = np.array_split(s_rp5min, 120)

## Staircase
d_up1 = process_file('processed_data/d_up_1_bw.csv')
d_up2 = process_file('processed_data/d_up_2_bw.csv')
d_up3 = process_file('processed_data/d_up_3_bw.csv')
d_up4 = process_file('processed_data/d_up_4_bw.csv')
d_up5 = process_file('processed_data/dlp_upstairs_1_bw.csv')
d_up6 = process_file('processed_data/dlp_upstairs_2_bw.csv')

d_down1 = process_file('processed_data/d_down_1_bw.csv')
d_down2 = process_file('processed_data/d_down_2_bw.csv')
d_down3 = process_file('processed_data/d_down_3_bw.csv')
d_down4 = process_file('processed_data/d_down_4_bw.csv')
d_down5 = process_file('processed_data/dlp_downstairs_1_bw.csv')
d_down6 = process_file('processed_data/dlp_downstairs_2_bw.csv')

m_up24_1_bw = process_file('processed_data\m_up24_1_bw.csv')
m_up24_2_bw = process_file('processed_data\m_up24_2_bw.csv')
m_up24_3_bw = process_file('processed_data\m_up24_3_bw.csv')
m_up24_4_bw = process_file('processed_data\m_up24_4_bw.csv')
m_up24_5_bw = process_file('processed_data\m_up24_5_bw.csv')

m_down24_1_bw = process_file('processed_data\m_down24_1_bw.csv')
m_down24_2_bw = process_file('processed_data\m_down24_2_bw.csv')
m_down24_3_bw = process_file('processed_data\m_down24_3_bw.csv')
m_down24_4_bw = process_file('processed_data\m_down24_4_bw.csv')
m_down24_5_bw = process_file('processed_data\m_down24_5_bw.csv')

d_up1['movement'] = 'upstairs'
d_up2['movement'] = 'upstairs'
d_up3['movement'] = 'upstairs'
d_up4['movement'] = 'upstairs'
d_up5['movement'] = 'upstairs'
d_up6['movement'] = 'upstairs'

d_up1['movement'] = 'downstairs'
d_up2['movement'] = 'downstairs'
d_up3['movement'] = 'downstairs'
d_up4['movement'] = 'downstairs'
d_up5['movement'] = 'downstairs'
d_up6['movement'] = 'downstairs'

m_up24_1_bw['movement'] = 'upstairs'
m_up24_2_bw['movement'] = 'upstairs'
m_up24_3_bw['movement'] = 'upstairs'
m_up24_4_bw['movement'] = 'upstairs'
m_up24_5_bw['movement'] = 'upstairs'

m_down24_1_bw['movement'] = 'downstairs'
m_down24_2_bw['movement'] = 'downstairs'
m_down24_3_bw['movement'] = 'downstairs'
m_down24_4_bw['movement'] = 'downstairs'
m_down24_5_bw['movement'] = 'downstairs'

d_up1_split = np.array_split(d_up1, 4)
d_up2_split = np.array_split(d_up2, 4)
d_up3_split = np.array_split(d_up3, 4)
d_up4_split = np.array_split(d_up4, 4)
d_up5_split = np.array_split(d_up5, 4)
d_up6_split = np.array_split(d_up6, 4)

d_down1_split = np.array_split(d_up1, 4)
d_down2_split = np.array_split(d_up2, 4)
d_down3_split = np.array_split(d_up3, 4)
d_down4_split = np.array_split(d_up4, 4)
d_down5_split = np.array_split(d_up5, 4)
d_down6_split = np.array_split(d_up6, 4)

m_up24_1_split = np.array_split(m_up24_1_bw, 4)
m_up24_2_split = np.array_split(m_up24_2_bw, 4)
m_up24_3_split = np.array_split(m_up24_3_bw, 4)
m_up24_4_split = np.array_split(m_up24_4_bw, 4)
m_up24_5_split = np.array_split(m_up24_5_bw, 4)

m_down24_1_split = np.array_split(m_down24_1_bw, 4)
m_down24_2_split = np.array_split(m_down24_2_bw, 4)
m_down24_3_split = np.array_split(m_down24_3_bw, 4)
m_down24_4_split = np.array_split(m_down24_4_bw, 4)
m_down24_5_split = np.array_split(m_down24_5_bw, 4)

tmove = []
tdata = []

for i in [d_walk_split, m_walk_split, s_walk_split,
            d_up1_split, d_up2_split, d_up3_split, d_up4_split, d_up5_split, d_up6_split, d_down1_split, d_down2_split, d_down3_split, d_down4_split, d_down5_split, d_down6_split,
            m_up24_1_split, m_up24_2_split, m_up24_3_split, m_up24_4_split, m_up24_5_split, m_down24_1_split, m_down24_2_split, m_down24_3_split, m_down24_4_split, m_down24_5_split]:
    for j in range(len(i) - 1):
        temp = i[j]

        if ((temp['movement'] == 'walking').all()):
            tmove.append('walking')
        if ((temp['movement'] == 'upstairs').all()):
            tmove.append('upstairs')
        if ((temp['movement'] == 'downstairs').all()):
            tmove.append('downstairs')

        temp['combined'] = temp['y'] + temp['z']
        tdata.append(np.asarray(temp['combined']))

training_data = pd.DataFrame(tmove)
training_data['joined'] = pd.Series(tdata, index = training_data.index)
training_data

training_data.rename(columns={0: 'movement'}, inplace=True)
training_data = pd.concat([training_data['movement'], training_data.pop('joined').apply(pd.Series)], axis=1).dropna(axis=1)

training_data


,movement,0,1,2,3,4,5,6,7,8,...,102,103,104,105,106,107,108,109,110,111
0,walking,1.240879,0.150242,-0.876929,-1.773043,-2.477482,-2.942631,-3.139237,-3.059383,-2.716308,...,3.877894,3.690279,3.371703,2.934023,2.389804,1.750149,1.025023,0.226857,-0.624618,-1.499339
1,walking,-4.788854,-4.683991,-4.374063,-3.880401,-3.228459,-2.444105,-1.552900,-0.582312,0.434396,...,2.000099,1.079813,0.074315,-0.982497,-2.050335,-3.080685,-4.017883,-4.802848,-5.378987,-5.699572
2,walking,-2.024978,-0.790810,0.479561,1.724391,2.878783,3.875747,4.648906,5.137365,5.293161,...,-5.684565,-5.921625,-5.853576,-5.484695,-4.839603,-3.958513,-2.891438,-1.693617,-0.422958,0.860961
3,walking,5.470634,5.256383,4.712832,3.878186,2.819325,1.624395,0.389637,-0.795473,-1.858937,...,-1.513420,-0.184376,1.139094,2.386459,3.488812,4.381133,5.005325,5.314568,5.279422,4.895158
4,walking,-1.527465,-2.498806,-3.256122,-3.761774,-3.995480,-3.955920,-3.660439,-3.141991,-2.444097,...,4.921624,4.559010,3.897111,2.988420,1.911133,0.756828,-0.384715,-1.437638,-2.345528,-3.071025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,downstairs,-0.988269,-2.122362,-3.128084,-3.948573,-4.541400,-4.881309,-4.961177,-4.790797,-4.394018,...,1.138947,-0.050253,-1.172120,-2.173535,-3.017441,-3.683114,-4.164468,-4.466655,-4.601598,-4.583356
419,downstairs,0.609541,1.986818,3.379457,4.694758,5.826160,6.669487,7.142123,7.199415,6.842893,...,-5.600998,-5.694508,-5.516422,-5.066065,-4.353342,-3.400739,-2.244867,-0.936970,0.457694,1.861728
420,downstairs,-0.945662,-1.528726,-2.053644,-2.483161,-2.792156,-2.970517,-3.023860,-2.972105,-2.846118,...,-4.433941,-4.092714,-3.458872,-2.581886,-1.529350,-0.380041,0.783754,1.883549,2.850392,3.627930
421,downstairs,-1.829955,-2.778048,-3.557972,-4.126523,-4.457902,-4.544385,-4.394924,-4.031828,-3.486414,...,4.630506,4.318612,3.758602,2.998296,2.095236,1.110979,0.105907,-0.865364,-1.757988,-2.539660


In [8]:
X = training_data.drop(columns = ['movement'])
y = training_data['movement']

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

bayes_model = make_pipeline(
    GaussianNB()
)

knn_model = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier(n_neighbors = 8)
)

nn_model = make_pipeline(
    StandardScaler(),
    MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = (16,8,4), activation = 'logistic', max_iter=100000)
)

dt_model = make_pipeline(
    StandardScaler(),
    DecisionTreeClassifier(max_depth = 125)
)

rf_model = make_pipeline(
    RandomForestClassifier(n_estimators = 1500 , max_depth = 3, min_samples_leaf = 5)
)

en_model = make_pipeline(
    VotingClassifier([
        ('nb', GaussianNB()),
        ('knn', KNeighborsClassifier(8)),
        ('tree1', DecisionTreeClassifier(max_depth= 20)),
        ('tree2', DecisionTreeClassifier(min_samples_leaf=4)),
        ('rf', RandomForestClassifier(n_estimators = 250 , max_depth = 15, min_samples_leaf = 2)),
    ('mlp', MLPClassifier(solver = 'lbfgs', hidden_layer_sizes = (16,8,6), activation = 'logistic', max_iter=100000) )])
    
)

print('bayes')
bayes_model.fit(X_train, y_train)
print(bayes_model.score(X_train, y_train))
print(bayes_model.score(X_valid, y_valid))
y_predicted = bayes_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('knn')
knn_model.fit(X_train, y_train)
print(knn_model.score(X_train, y_train))
print(knn_model.score(X_valid, y_valid))
y_predicted = knn_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('neural net')
nn_model.fit(X_train, y_train)
print(nn_model.score(X_train, y_train))
print(nn_model.score(X_valid, y_valid))
y_predicted = nn_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('decision tree')
dt_model.fit(X_train, y_train)
print(dt_model.score(X_train, y_train))
print(dt_model.score(X_valid, y_valid))
y_predicted = dt_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('random forest')
rf_model.fit(X_train, y_train)
print(rf_model.score(X_train, y_train))
print(rf_model.score(X_valid, y_valid))
y_predicted = rf_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('ensemble')
en_model.fit(X_train, y_train)
print(en_model.score(X_train, y_train))
print(en_model.score(X_valid, y_valid))
y_predicted = en_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

bayes
0.7003154574132492
0.7264150943396226
              precision    recall  f1-score   support

  downstairs       0.37      0.83      0.51        12
    upstairs       0.33      1.00      0.50         5
     walking       0.97      0.70      0.81        89

    accuracy                           0.73       106
   macro avg       0.56      0.84      0.61       106
weighted avg       0.87      0.73      0.76       106


knn
0.9337539432176656
0.9339622641509434
              precision    recall  f1-score   support

  downstairs       1.00      0.83      0.91        12
    upstairs       0.00      0.00      0.00         5
     walking       0.93      1.00      0.96        89

    accuracy                           0.93       106
   macro avg       0.64      0.61      0.62       106
weighted avg       0.89      0.93      0.91       106


neural net
0.9968454258675079
0.839622641509434
              precision    recall  f1-score   support

  downstairs       0.47      0.67      0.55    